In [1]:
!pip install git+https://github.com/dynamiqs/dynamiqs.git
import dynamiqs as dq
import numpy as np
import qutip as qt
import torch

  Cloning https://github.com/dynamiqs/dynamiqs.git to /tmp/pip-req-build-n447pmux
  Running command git clone --filter=blob:none --quiet https://github.com/dynamiqs/dynamiqs.git /tmp/pip-req-build-n447pmux
  Resolved https://github.com/dynamiqs/dynamiqs.git to commit dbd157063a00c1ec2342fb74a57d3e77e243a7e1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 27.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.5/531.5 kB 53.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dynamiqs: filename=dynamiqs-0.1.0-py2.py3-none-any.whl size=79265 sha256=ffbf200fd21921562fa9cd06abda56cc399e79d217d70b16fe1259229b3bc801
  Stored in directory: /tmp/pip-ephem-wheel-cache-1fh6gc0p

/usr/local/lib/python3.10/dist-packages/qutip/__init__.py:66: UserWarning: The new version of Cython, (>= 3.0.0) is not supported.
  warnings.warn(


In [48]:
## Spin parameters
h    = 6.6260693e-34       # J.s
mu_N = 5.0507836991e-27    # Nuclear magneton in J/T
g_N  = 0.2355695           # g nuclear 183W (INDC International Nuclear Data Committee)
mu_I = mu_N*g_N            # Nuclear magneton * g nuclear 183W  in J/T
tau = 2*torch.pi           # two pi

# Magnetic field
B0 = 0.447 # T
theta = 0*torch.pi/180 # rad
phi = 0*torch.pi/180 # rad
B_field = np.array([B0*np.sin(theta)*np.cos(phi), B0*np.sin(theta)*np.sin(phi), B0*np.cos(theta)])

# omega_L = 2*torch.pi*mu_I/h*1e-6*B0 # rad/us
omega_L = 790.40/1000*2*torch.pi

# Hyperfine constants
A = tau * 1e-3 * (-35.2)
B = tau * 1e-3 * 70.4

In [64]:
S = 1/2
nS = int(2*S+1)
Si = qt.qeye(nS)

I = 1/2
nI = int(2*I+1)
Ii = qt.qeye(nI)

Sx, Sy, Sz = qt.jmat(S)
Sp, Sm = qt.spin_Jp(S), qt.spin_Jm(S)

SxI, SyI, SzI = (qt.tensor(op, Ii) for op in (Sx,Sy,Sz))
SpI, SmI = (qt.tensor(op, Ii) for op in (Sp,Sm))

Ix, Iy, Iz = qt.jmat(I)
Ip, Im = qt.spin_Jp(I), qt.spin_Jm(I)

IIx, IIy, IIz = (qt.tensor(Si, op) for op in (Ix,Iy,Iz))
IIp, IIm = (qt.tensor(Si, op) for op in (Ip,Im))

H_NZ = qt.tensor(Si, omega_L*Iz)

H_DD = A*qt.tensor(Sz,Iz) + B*qt.tensor(Sz, Ix)

H0 = H_NZ + H_DD

In [65]:
K00 = H0.eigenstates()[1][0]
K10 = H0.eigenstates()[1][1]
K01 = H0.eigenstates()[1][3]
K11 = H0.eigenstates()[1][2]

In [66]:
torch.set_default_device('cuda')

if type(H0) is not torch.Tensor:
  H0 = dq.to_tensor(H0, dtype=torch.complex64, device='cuda:0')

if type(K00) is not torch.Tensor:
  K00, K10, K01, K11 = (dq.to_tensor(ket, dtype=torch.complex64, device='cuda:0') for ket in (K00, K10, K01, K11))

if type(SxI) is not torch.Tensor:
  SxI, SyI, SzI, SpI, SmI = (dq.to_tensor(op, dtype=torch.complex64, device='cuda:0') for op in (SxI, SyI, SzI, SpI, SmI))

if type(IIx) is not torch.Tensor:
  IIx, IIy, IIz, IIp, IIm = (dq.to_tensor(op, dtype=torch.complex64, device='cuda:0') for op in (IIx, IIy, IIz, IIp, IIm))

In [68]:
H0.device

device(type='cuda', index=0)

In [72]:
omega_LO = 0

def square_pulse(t, pulse_duration = 0, amplitude = 0, t0 = 0):
    if t > (t0 - pulse_duration) and t < (t0 + pulse_duration):
        return amplitude
    else:
        return 0

def square_pulse_p(t, detuning = 0, phase = 0, **args):
  return square_pulse(t, **args) * np.exp(+1j*((detuning+omega_LO)*t + phase))

def square_pulse_m(t, detuning = 0, phase = 0, **args):
  return square_pulse(t, **args) * np.exp(-1j*((detuning+omega_LO)*t + phase))

In [141]:
args = {
    't0': 0,
    'pulse_duration': 100,
    'amplitude': 0.2,
    'phase': 0,
    'detuning': -0.1
}

In [162]:
torch.set_default_device('cpu')
H0, SpI, SmI = H0.cpu(), SpI.cpu(), SmI.cpu()

batch_n = 500
detuning = A/2*torch.linspace(-4, 4, batch_n)
args = {
    't0': 0,
    'pulse_duration': 100,
    'amplitude': 0.2,
    'phase': 0,
    'detuning': detuning
}

def square_pulse(t, pulse_duration = 0, amplitude = 0, t0 = 0):
    if t > (t0 - pulse_duration) and t < (t0 + pulse_duration):
        return amplitude
    else:
        return 0

def square_pulse_p(t, detuning = 0, phase = 0, **args):
  return square_pulse(t, **args) * torch.exp(+1j*((detuning)*t + phase))

def square_pulse_m(t, detuning = 0, phase = 0, **args):
  return square_pulse(t, **args) * torch.exp(-1j*((detuning)*t + phase))

def H_args(t, batch_n, **args):
  return H0.repeat(batch_n,1,1) + square_pulse_p(t, **args).view(-1,1,1)*SpI.repeat(batch_n,1,1) + square_pulse_m(t, **args).view(-1,1,1)*SmI.repeat(batch_n,1,1)

H = dq.totime(lambda t: H_args(t, batch_n, **args), dtype=torch.complex64)

tlist = np.linspace(0, 50, 1000)
psi0 = K10.cpu()

result = dq.sesolve(H, psi0, tlist, exp_ops = [SpI@SmI, IIp@IIm])
print(result)

|██████████| 100.0% - time 00:05/00:00

==== Result ====
Solver     : Dopri5
Start      : 2024-02-11 01:04:24
End        : 2024-02-11 01:04:30
Total time : 5.47 s
States     : Tensor (500, 1000, 4, 1) | 15.26 Mb
Expects    : Tensor (500, 2, 1000) | 7.63 Mb


In [119]:
result.expects[0]

tensor([1.0000+0.j, 0.9898+0.j, 0.9598+0.j, 0.9111+0.j, 0.8457+0.j, 0.7663+0.j, 0.6761+0.j,
        0.5788+0.j, 0.4782+0.j, 0.3786+0.j, 0.2839+0.j, 0.1980+0.j, 0.1244+0.j, 0.0660+0.j,
        0.0253+0.j, 0.0039+0.j, 0.0026+0.j, 0.0216+0.j, 0.0601+0.j, 0.1163+0.j, 0.1883+0.j,
        0.2728+0.j, 0.3667+0.j, 0.4659+0.j, 0.5666+0.j, 0.6645+0.j, 0.7558+0.j, 0.8367+0.j,
        0.9040+0.j, 0.9548+0.j, 0.9872+0.j, 0.9998+0.j, 0.9922+0.j, 0.9645+0.j, 0.9180+0.j,
        0.8545+0.j, 0.7767+0.j, 0.6876+0.j, 0.5909+0.j, 0.4905+0.j, 0.3906+0.j, 0.2950+0.j,
        0.2079+0.j, 0.1326+0.j, 0.0722+0.j, 0.0293+0.j, 0.0054+0.j, 0.0017+0.j, 0.0182+0.j,
        0.0544+0.j, 0.1086+0.j, 0.1787+0.j, 0.2619+0.j, 0.3548+0.j, 0.4536+0.j, 0.5543+0.j,
        0.6528+0.j, 0.7451+0.j, 0.8275+0.j, 0.8966+0.j, 0.9496+0.j, 0.9843+0.j, 0.9994+0.j,
        0.9942+0.j, 0.9689+0.j, 0.9246+0.j, 0.8631+0.j, 0.7868+0.j, 0.6989+0.j, 0.6030+0.j,
        0.5028+0.j, 0.4026+0.j, 0.3063+0.j, 0.2179+0.j, 0.1410+0.j, 0.0787+0.j, 

In [163]:
torch.set_default_device('cuda')
H0, SpI, SmI = H0.cuda(), SpI.cuda(), SmI.cuda()

batch_n = 500
detuning = A/2*torch.linspace(-4, 4, batch_n)
args = {
    't0': 0,
    'pulse_duration': 100,
    'amplitude': 0.2,
    'phase': 0,
    'detuning': detuning
}

omega_LO = 0

def square_pulse(t, pulse_duration = 0, amplitude = 0, t0 = 0):
    if t > (t0 - pulse_duration) and t < (t0 + pulse_duration):
        return amplitude
    else:
        return 0

def square_pulse_p(t, detuning = 0, phase = 0, **args):
  return square_pulse(t, **args) * torch.exp(+1j*((detuning+omega_LO)*t + phase))

def square_pulse_m(t, detuning = 0, phase = 0, **args):
  return square_pulse(t, **args) * torch.exp(-1j*((detuning+omega_LO)*t + phase))

def H_args(t, batch_n, **args):
  return H0.repeat(batch_n,1,1) + square_pulse_p(t, **args).view(-1,1,1)*SpI.repeat(batch_n,1,1) + square_pulse_m(t, **args).view(-1,1,1)*SmI.repeat(batch_n,1,1)

H = dq.totime(lambda t: H_args(t, batch_n, **args), dtype=torch.complex64)

tlist = np.linspace(0, 50, 1000)
psi0 = K10.cuda()

result = dq.sesolve(H, psi0, tlist, exp_ops = [SpI@SmI, IIp@IIm])
print(result)

|██████████| 100.0% - time 00:05/00:00

==== Result ====
Solver     : Dopri5
Start      : 2024-02-11 01:04:42
End        : 2024-02-11 01:04:48
Total time : 6.00 s
States     : Tensor (500, 1000, 4, 1) | 15.26 Mb
Expects    : Tensor (500, 2, 1000) | 7.63 Mb


In [151]:
batch_n = 50
detuning = A/2*torch.linspace(-4, 4, batch_n)
t=5
phase=0
torch.exp(-1j*((detuning+omega_LO)*t + phase))

tensor([-0.5979-0.8016j, -0.5232-0.8522j, -0.4443-0.8959j, -0.3617-0.9323j,
        -0.2762-0.9611j, -0.1884-0.9821j, -0.0991-0.9951j, -0.0090-1.0000j,
         0.0812-0.9967j,  0.1707-0.9853j,  0.2589-0.9659j,  0.3449-0.9386j,
         0.4281-0.9037j,  0.5078-0.8615j,  0.5834-0.8122j,  0.6543-0.7563j,
         0.7198-0.6942j,  0.7794-0.6265j,  0.8327-0.5537j,  0.8793-0.4764j,
         0.9186-0.3951j,  0.9505-0.3107j,  0.9746-0.2238j,  0.9908-0.1350j,
         0.9990-0.0451j,  0.9990+0.0451j,  0.9908+0.1350j,  0.9746+0.2238j,
         0.9505+0.3107j,  0.9186+0.3951j,  0.8793+0.4764j,  0.8327+0.5537j,
         0.7794+0.6265j,  0.7198+0.6942j,  0.6543+0.7563j,  0.5834+0.8122j,
         0.5078+0.8615j,  0.4281+0.9037j,  0.3449+0.9386j,  0.2589+0.9659j,
         0.1707+0.9853j,  0.0812+0.9967j, -0.0090+1.0000j, -0.0991+0.9951j,
        -0.1884+0.9821j, -0.2762+0.9611j, -0.3617+0.9323j, -0.4443+0.8959j,
        -0.5232+0.8522j, -0.5979+0.8016j])

In [157]:
H0.repeat(5,1,1)

tensor([[[ 2.4278+0.j,  0.1106+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.1106+0.j, -2.4278+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.0000+0.j,  0.0000+0.j,  2.5384+0.j, -0.1106+0.j],
         [ 0.0000+0.j,  0.0000+0.j, -0.1106+0.j, -2.5384+0.j]],

        [[ 2.4278+0.j,  0.1106+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.1106+0.j, -2.4278+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.0000+0.j,  0.0000+0.j,  2.5384+0.j, -0.1106+0.j],
         [ 0.0000+0.j,  0.0000+0.j, -0.1106+0.j, -2.5384+0.j]],

        [[ 2.4278+0.j,  0.1106+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.1106+0.j, -2.4278+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.0000+0.j,  0.0000+0.j,  2.5384+0.j, -0.1106+0.j],
         [ 0.0000+0.j,  0.0000+0.j, -0.1106+0.j, -2.5384+0.j]],

        [[ 2.4278+0.j,  0.1106+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.1106+0.j, -2.4278+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.0000+0.j,  0.0000+0.j,  2.5384+0.j, -0.1106+0.j],
         [ 0.0000+0.j,  0.0000+0.j, -0.1106+0.j, 

In [125]:
torch.stack([H0, H0])+torch.stack([SpI, SpI])

tensor([[[ 2.4278+0.j,  0.1106+0.j,  1.0000+0.j,  0.0000+0.j],
         [ 0.1106+0.j, -2.4278+0.j,  0.0000+0.j,  1.0000+0.j],
         [ 0.0000+0.j,  0.0000+0.j,  2.5384+0.j, -0.1106+0.j],
         [ 0.0000+0.j,  0.0000+0.j, -0.1106+0.j, -2.5384+0.j]],

        [[ 2.4278+0.j,  0.1106+0.j,  1.0000+0.j,  0.0000+0.j],
         [ 0.1106+0.j, -2.4278+0.j,  0.0000+0.j,  1.0000+0.j],
         [ 0.0000+0.j,  0.0000+0.j,  2.5384+0.j, -0.1106+0.j],
         [ 0.0000+0.j,  0.0000+0.j, -0.1106+0.j, -2.5384+0.j]]])

In [136]:
torch.stack([H0, H0])

tensor([[[ 2.4278+0.j,  0.1106+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.1106+0.j, -2.4278+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.0000+0.j,  0.0000+0.j,  2.5384+0.j, -0.1106+0.j],
         [ 0.0000+0.j,  0.0000+0.j, -0.1106+0.j, -2.5384+0.j]],

        [[ 2.4278+0.j,  0.1106+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.1106+0.j, -2.4278+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.0000+0.j,  0.0000+0.j,  2.5384+0.j, -0.1106+0.j],
         [ 0.0000+0.j,  0.0000+0.j, -0.1106+0.j, -2.5384+0.j]]])

In [140]:
torch.tensor([1,2]).view(-1, 1, 1)*torch.stack([H0, H0])

tensor([[[ 2.4278+0.j,  0.1106+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.1106+0.j, -2.4278+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.0000+0.j,  0.0000+0.j,  2.5384+0.j, -0.1106+0.j],
         [ 0.0000+0.j,  0.0000+0.j, -0.1106+0.j, -2.5384+0.j]],

        [[ 4.8556+0.j,  0.2212+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.2212+0.j, -4.8556+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.0000+0.j,  0.0000+0.j,  5.0768+0.j, -0.2212+0.j],
         [ 0.0000+0.j,  0.0000+0.j, -0.2212+0.j, -5.0768+0.j]]])

In [132]:
# Define the 2D tensor
tensor_2d = torch.tensor([[1, 2], [3, 4]])

# Define the 1D tensor
tensor_1d = torch.tensor([1, 2])

# Reshape tensor_1d to make it compatible for broadcasting with tensor_2d
# We add an extra dimension at the end to make it a column vector
tensor_1d_reshaped = tensor_1d.view(-1, 1)

# Perform element-wise multiplication with broadcasting
result = tensor_2d * tensor_1d_reshaped.T

print("Result of multiplication:\n", result)

Result of multiplication:
 tensor([[1, 4],
        [3, 8]])


In [138]:
tensor_1d_reshaped

tensor([[1],
        [2]])